In [39]:
using LinearAlgebra

function cross_prod_matrix( k )
    return [ 0  -k[3] k[2];  
            k[3] 0   -k[1]; 
           -k[2] k[1] 0 ]
end 

cross_prod_matrix (generic function with 1 method)

In [40]:
function rot_matrix_3d_transform( k, theta )
    mat0 = [ 1 0          0; 
             0 cos(theta) -sin(theta); 
             0 sin(theta) cos( theta )]
    
    if isapprox( k[1], 0.0 ) && isapprox( k[2], 0.0 )
        u = [ 0.0, 0.0, 1.0 ]
    else 
        u = [ k[2], -k[1], 0 ]
        u /= norm( u )
    end 
    
    v = cross( k, u )
    trans = hcat( k, u, v )
    return trans*mat0*transpose( trans )
end 

rot_matrix_3d_transform (generic function with 1 method)

In [41]:
function rot_matrix_3d( k, theta )
    
    K = cross_prod_matrix( k )
    return I + sin(theta)K + (1-cos(theta))K^2
end 

rot_matrix_3d (generic function with 1 method)

In [42]:
rot_x_theta( theta ) = rot_matrix_3d( [1.0, 0.0, 0.0], theta )
rot_y_theta( theta ) = rot_matrix_3d( [0.0, 1.0, 0.0], theta )
rot_z_theta( theta ) = rot_matrix_3d( [0.0, 0.0, 1.0], theta )    

rot_z_theta (generic function with 1 method)

In [235]:
function rand_rot_3d()

    k = [ rand()- .5 for x in 1:3 ]
    k = k/norm(k)
    theta = 2*pi*rand()
    return rot_matrix_3d( k, theta )
end

rand_rot_3d (generic function with 1 method)

In [236]:
function params_rotation_3d( R )
   
    theta = acos( (R[1,1] + R[2,2] + R[3,3] -1 )/2 )
    k1 = (R[3,2]-R[2,3])/(2*sin(theta))
    k2 = (R[1,3]-R[3,1])/(2*sin(theta))
    k3 = (R[2,1]-R[1,2])/(2*sin(theta))
    
    return [k1, k2, k3], theta
end

params_rotation_3d (generic function with 1 method)

In [261]:
function zxz_angles( R ) 
    
    theta = acos( R[3,3] )
    
    phi = pi/2#asin( R[3,1]/sin(theta) ); 
    if !isapprox( R[3,2], sin(theta)*cos(phi))
            phi = pi-phi
    end
    
    psi = -pi/2#asin( R[1,3]/sin(theta))
    if !isapprox( R[2,3], -cos(psi)*sin(theta)) 
        psi = pi-psi
    end 
    
    return phi, theta, psi
end

zxz_angles (generic function with 1 method)

In [240]:
function zyx_angles( R )
    theta = asin( -R[3,1] )
    phi = asin( R[3,2]/cos(theta))
    
    if !isapprox( cos(phi)*cos(theta), R[3,3] )
        phi = pi-phi
    end 
    
    psi = asin( R[2,1]/cos(theta))
    
    if !isapprox( cos(theta)*cos(psi), R[1,1])
        psi = pi-psi
    end
    
    return phi, theta, psi
end

zyx_angles (generic function with 1 method)

In [241]:
R = rand_rot_3d()

3×3 Matrix{Float64}:
 -0.715848  0.628278   0.304678
  0.630283  0.393617   0.669185
  0.300508  0.671068  -0.677763

In [244]:
phi, theta, psi = zyx_angles( R )
rot_z_theta( psi )*rot_y_theta( theta )*rot_x_theta( phi )

3×3 Matrix{Float64}:
 -0.715848  0.628278   0.304678
  0.630283  0.393617   0.669185
  0.300508  0.671068  -0.677763

In [246]:
phi, theta, psi = zxz_angles( R )
rot_z_theta( psi )*rot_x_theta( theta )*rot_z_theta( phi )

3×3 Matrix{Float64}:
 -0.715848  0.628278   0.304678
  0.630283  0.393617   0.669185
  0.300508  0.671068  -0.677763

In [250]:
r = rot_y_theta( 2pi*rand() )

3×3 Matrix{Float64}:
 0.90041   0.0  -0.435043
 0.0       1.0   0.0
 0.435043  0.0   0.90041

In [262]:
phi, theta, psi = zxz_angles( r )
rot_z_theta( psi )*rot_x_theta( theta )*rot_z_theta( phi )

3×3 Matrix{Float64}:
  0.90041      3.2201e-16   -0.435043
 -3.10953e-16  1.0           9.65989e-17
  0.435043     4.82994e-17   0.90041

In [253]:
isapprox(1.0000000000000002, 1 )

true

In [263]:
phi, theta, psi

(1.5707963267948966, 0.450085557520617, 4.71238898038469)

In [264]:
pi/2

1.5707963267948966